___
# Supply Planning - Summary


### To view the logic for the detail supply planning alogorithm please connect toe the below link

[Detail Supply Planning Algorithm](SupplyPlanning.ipynb)

___
# Literature on Supply Planning / MRP

The goal of using an MRP strategy is to run your operations in a manner which improves your customer response without over
producing and having a lot of extra stock.

Two main objectives that MRP strategies try to solve are inventory reduction or lead time reduction.

One way to classifiy the materials you use for MRP is : 1) Cheap materials (Use consumption based methods - reorder point planning) 2)Long lead time materials or expensive materials needs other starategies

What do you need to consider while choosing a strategy :

1. Level of the BOM where demand management is done
    - Finished Goods - Component Levele - Phantom level

2. What point to produce final assembly
    - Make to stock - Make to order
    
3. What point yo procure components
    - Component in stock - Purchase stock after sale
  
4. Should current stock levels influence my production and procurement

In The below planning process we formulate a Net Requirements Planning from distributor point of view which assumes unconstraint requirements generation.

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil.relativedelta import relativedelta
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

### Loading the results of planning from Supply Planning Sheet

In [37]:
data = pd.read_csv("result.csv")
data['Year'] = data['Year'].astype('str')

In [38]:
data.head()

,Plan,SKU,Date,Agency,Brand,Desc...4,Desc...5,Super.Category,Literage,Pack,...,Avg.CA,SD.Stocks,StockDays,OOS.CA,Time.Fence,Year,Month,Excess.CA,Scheduled.CA,ABClass
0,Region 1,Product 1002,2021-04-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,0.67,1.16,0.00,0.00,2021-09-01,2021,Apr,0.00,0.00,C
1,Region 1,Product 1002,2021-05-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,1.00,1.73,0.00,0.00,2021-09-01,2021,May,0.00,0.00,C
2,Region 1,Product 1002,2021-06-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,1.67,2.89,0.00,0.00,2021-09-01,2021,Jun,0.00,0.00,C
3,Region 1,Product 1002,2021-07-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,2.33,4.04,0.00,3.00,2021-09-01,2021,Jul,0.00,11.93,C
4,Region 1,Product 1002,2021-08-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,2.33,4.04,0.00,2.00,2021-09-01,2021,Aug,0.00,1.42,C


## Lets summerize the data (Taking only the first 3 months Jul, Aug, Sep)

### The below shows the number of SKU's planned for each Region / Category / ABC Class

In [39]:
data['Date'] = pd.to_datetime(data['Date'])
current_date = datetime.today()
future_date = current_date + relativedelta(months=3)
datasummary = data[(data['Date'] < future_date) & (data['Date'] > current_date - relativedelta(days=21))] 
val = ['Forecast.CA','OOS.CA']
col = ['Year','Month']
idx = ['Plan','Super.Category','ABClass']
datasummary.pivot_table(values= 'SKU', columns=['Year','Month'], index=idx, aggfunc=lambda x: len(x.unique()), margins='all')

Year                             2021                     All
Month                             Aug   Jul   Oct   Sep      
Plan     Super.Category ABClass                              
Region 1 Category 1     A         104   104   104   104   104
                        B         115   115   115   115   115
                        C         285   285   285   285   285
         Category 2     A          61    61    61    61    61
                        B         114   114   114   114   114
                        C         494   494   494   494   494
         Category 3     A          10    10    10    10    10
                        B          16    16    16    16    16
                        C          68    68    68    68    68
         Category 4     A          11    11    11    11    11
                        B           8     8     8     8     8
                        C          22    22    22    22    22
Region 2 Category 1     A         226   226   226   226   226
                        B         317   317   317   317   317
                        C         841   841   841   841   841
         Category 2     A          82    82    82    82    82
                        B         110   110   110   110   110
                        C         457   457   457   457   457
         Category 3     A          23    23    23    23    23
                        B          39    39    39    39    39
                        C         125   125   125   125   125
         Category 4     A          16    16    16    16    16
                        B          20    20    20    20    20
                        C          60    60    60    60    60
Region 3 Category 1     A         114   114   114   114   114
                        B         186   186   186   186   186
                        C         368   368   368   368   368
         Category 2     A          36    36    36    36    36
                        B          71    71    71    71    71
                        C         381   381   381   381   381
         Category 3     A          21    21    21    21    21
                        B          25    25    25    25    25
                        C         122   122   122   122   122
         Category 4     A           8     8     8     8     8
                        B          10    10    10    10    10
                        C          13    13    13    13    13
Region 4 Category 1     A         147   147   147   147   147
                        B         237   237   237   237   237
                        C         590   590   590   590   590
         Category 2     A          48    48    48    48    48
                        B         125   125   125   125   125
                        C         633   633   633   633   633
         Category 3     A          13    13    13    13    13
                        B          32    32    32    32    32
                        C         129   129   129   129   129
         Category 4     A          13    13    13    13    13
                        B          10    10    10    10    10
                        C          17    17    17    17    17
All                              3810  3810  3810  3810  3810

In [40]:
datasummary.head()

,Plan,SKU,Date,Agency,Brand,Desc...4,Desc...5,Super.Category,Literage,Pack,...,Avg.CA,SD.Stocks,StockDays,OOS.CA,Time.Fence,Year,Month,Excess.CA,Scheduled.CA,ABClass
3,Region 1,Product 1002,2021-07-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,2.33,4.04,0.00,3.00,2021-09-01,2021,Jul,0.00,11.93,C
4,Region 1,Product 1002,2021-08-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,2.33,4.04,0.00,2.00,2021-09-01,2021,Aug,0.00,1.42,C
5,Region 1,Product 1002,2021-09-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,2.33,4.04,52.00,0.00,2021-09-01,2021,Sep,0.00,7.00,C
6,Region 1,Product 1002,2021-10-01,Supplier 89,Band 216,Desc 1002,Desc 1013,Category 2,4.50,6,...,4.00,6.93,52.00,0.00,2021-09-01,2021,Oct,0.00,1.42,C
20,Region 1,Product 1003,2021-07-01,Supplier 89,Band 216,Desc 1003,Desc 1014,Category 2,4.50,6,...,9.33,16.18,51.43,0.00,2021-09-01,2021,Jul,0.00,30.33,C


## Now lets look at the summaries of Forecasted Volume and Out of Stock Volume

In [41]:
val = ['Forecast.CA','OOS.CA']
col = ['Date']
idx = ['Plan','Super.Category','ABClass']
summary1 = datasummary.pivot_table(values= val, columns=col, index=idx, aggfunc={'Forecast.CA':'sum','OOS.CA':'sum'}).reorder_levels([1,0], axis=1).sort_index(axis=1)
summary1.head(12)


Date                             2021-07-01           2021-08-01           \
                                Forecast.CA   OOS.CA Forecast.CA   OOS.CA   
Plan     Super.Category ABClass                                             
Region 1 Category 1     A          1,267.00   252.42    1,910.00   645.92   
                        B            531.00   159.50      544.00   133.67   
                        C            595.00   160.37      639.00   152.75   
         Category 2     A         19,337.00 8,272.58   34,084.00 3,755.50   
                        B          7,076.00 2,491.67    8,465.00 1,618.00   
                        C          4,772.00 1,057.88    5,454.00 1,087.42   
         Category 3     A          8,332.00    99.00   13,057.00 1,656.12   
                        B          3,555.00   632.00    4,234.00 1,532.00   
                        C          3,587.00 2,470.00    4,688.00 3,168.96   
         Category 4     A            129.00    32.00      211.00    59.00   
                        B             34.00     1.08       42.00    11.67   
                        C             45.00     2.00       51.00     9.67   

Date                             2021-09-01         2021-10-01         
                                Forecast.CA OOS.CA Forecast.CA OOS.CA  
Plan     Super.Category ABClass                                        
Region 1 Category 1     A          2,226.00 711.17    2,211.00  39.00  
                        B            515.00  60.92      713.00   0.00  
                        C            593.00  40.33      676.00   3.00  
         Category 2     A         32,289.00 593.75   30,748.00   0.00  
                        B          8,184.00 560.25    8,469.00 119.00  
                        C          5,368.00 348.25    6,037.00  34.17  
         Category 3     A         13,058.00   0.00   14,159.00   0.00  
                        B          5,325.00 235.00    4,266.00   0.00  
                        C          4,409.00 399.00    4,585.00 301.00  
         Category 4     A            249.00   0.00      238.00   0.00  
                        B             46.00   0.00       52.00   0.00  
                        C             46.00   0.00       97.00   0.00

## Now lets look into the SKU's that have high out of stock volumes

In [42]:
#datasummary.groupby(['Plan','Super.Category','SKU','Year','Month'])['OOS.CA'].agg(np.sum).reset_index()
oos = datasummary.pivot_table(index=['Plan','Super.Category','SKU'], columns=['Month'], values='OOS.CA', aggfunc='sum').reset_index()
oos['Total'] = oos['Jul'] + oos['Aug'] + oos['Sep'] + oos['Oct']
oos.sort_values(['Plan','Total'],ascending=False).head()

Month,Plan,Super.Category,SKU,Aug,Jul,Oct,Sep,Total
6820,Region 4,Category 3,Product 28,0.00,"8,021.63","3,418.00",0.00,"11,439.63"
5997,Region 4,Category 2,Product 1081,820.00,"3,288.00",0.00,0.00,"4,108.00"
6834,Region 4,Category 3,Product 31,0.00,0.00,"2,400.00",0.00,"2,400.00"
6807,Region 4,Category 3,Product 23,316.00,520.00,516.00,321.00,"1,673.00"
6848,Region 4,Category 3,Product 35,451.00,410.00,0.00,483.00,"1,344.00"


### Lets look Region 4 SKU: Product 31	to understand why there is an availability issue

### The below explaning will give you an idea on how the horizontal plan was constructed for the SKU

#### Sales.CA is the month to date sales volume

![](Capture.PNG)

In [43]:
data['Date'] = pd.to_datetime(data['Date'])
current_date = datetime.today()
future_date = current_date + relativedelta(months=7)
datadetails = data[(data['Date'] < future_date) & (data['Date'] > current_date - relativedelta(days=16))] 
ids = ['Plan','SKU','LT','SD','ABClass','Date']
val = ['Beginning.Inv.CA','Transfer.CA','Purchase.CA','Planned.CA','Forecast.CA',
       'Sales.CA','Closing.inv.CA','OOS.CA','Exs.Consump.CA','Scheduled.CA','StockDays','Excess.CA']
datadetails = datadetails.melt(id_vars=ids, value_vars=val)
ids = ['Plan','SKU','LT','SD','ABClass','variable']
datadetails = datadetails.pivot_table(values='value', index=ids, columns = 'Date', aggfunc=np.sum)
a = datadetails.xs(['Region 4','Product 31'],level=['Plan','SKU']).reset_index()
sorter = dict(zip(val, range(len(val))))
a['Tm_Rank'] = a['variable'].map(sorter)
a.sort_values(['Tm_Rank'], ascending = True).drop('Tm_Rank', 1).set_index(['LT','SD','ABClass','variable'])

Date                             2021-08-01 00:00:00  2021-09-01 00:00:00  \
LT  SD ABClass variable                                                     
115 45 B       Beginning.Inv.CA             6,376.00             8,079.00   
               Transfer.CA                      0.00                 0.00   
               Purchase.CA                  5,203.00                 0.00   
               Planned.CA                       0.00                 0.00   
               Forecast.CA                  3,500.00             4,775.00   
               Sales.CA                         0.00                 0.00   
               Closing.inv.CA               8,079.00             3,304.00   
               OOS.CA                           0.00                 0.00   
               Exs.Consump.CA                   0.00                 0.00   
               Scheduled.CA                 3,002.50             4,501.50   
               StockDays                       52.01                19.85   
               Excess.CA                        0.00                 0.00   

Date                             2021-10-01 00:00:00  2021-11-01 00:00:00  \
LT  SD ABClass variable                                                     
115 45 B       Beginning.Inv.CA             3,304.00                 0.00   
               Transfer.CA                      0.00                 0.00   
               Purchase.CA                      0.00                 0.00   
               Planned.CA                       0.00            10,752.50   
               Forecast.CA                  5,704.00             4,501.00   
               Sales.CA                         0.00                 0.00   
               Closing.inv.CA                   0.00             6,251.50   
               OOS.CA                       2,400.00                 0.00   
               Exs.Consump.CA                   0.00                 0.00   
               Scheduled.CA                 3,004.50             2,820.00   
               StockDays                        0.00                45.00   
               Excess.CA                        0.00                 0.00   

Date                             2021-12-01 00:00:00  2022-01-01 00:00:00  \
LT  SD ABClass variable                                                     
115 45 B       Beginning.Inv.CA             6,251.50             5,753.00   
               Transfer.CA                      0.00                 0.00   
               Purchase.CA                      0.00                 0.00   
               Planned.CA                   3,002.50             4,501.50   
               Forecast.CA                  3,501.00             4,501.00   
               Sales.CA                         0.00                 0.00   
               Closing.inv.CA               5,753.00             5,753.50   
               OOS.CA                           0.00                 0.00   
               Exs.Consump.CA                   0.00                 0.00   
               Scheduled.CA                 2,706.50             2,290.50   
               StockDays                       45.00                45.00   
               Excess.CA                        0.00                 0.00   

Date                             2022-02-01 00:00:00  
LT  SD ABClass variable                               
115 45 B       Beginning.Inv.CA             5,753.50  
               Transfer.CA                      0.00  
               Purchase.CA                      0.00  
               Planned.CA                   3,004.50  
               Forecast.CA                  3,504.00  
               Sales.CA                         0.00  
               Closing.inv.CA               5,254.00  
               OOS.CA                           0.00  
               Exs.Consump.CA                   0.00  
               Scheduled.CA                 2,165.50  
               StockDays                       45.00  
               Excess.CA                        